In [10]:
import pandas as pd
import os
from IPython.display import display

# Step 1: Load raw and incremental data
raw = pd.read_csv("data/raw_data.csv")
inc = pd.read_csv("data/incremental_data.csv")

# Show raw data before transformation
print("Original Raw Data:")
display(raw.head())

print("\nOriginal Incremental Data:")
display(inc.head())

# Step 2: Define transformation function
def transform(df):
    df = df.copy()
    
    # Remove duplicates
    df.drop_duplicates(inplace=True)

    # Drop rows missing customer_name, product, or order_date
    df.dropna(subset=['customer_name', 'product', 'order_date'], inplace=True)

    # Fill missing quantity and unit_price with 0
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(0)
    df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce').fillna(0)

    # Add total_price
    df['total_price'] = df['quantity'] * df['unit_price']

    # Convert order_date to datetime
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

    # Add customer tier column
    df['tier'] = pd.cut(df['total_price'],
                        bins=[-1, 500, 1500, 10000],
                        labels=["Low", "Medium", "High"])
    return df

# Step 3: Apply transformations
clean_raw = transform(raw)
clean_inc = transform(inc)

# Show transformed data
print("\nTransformed Raw Data:")
display(clean_raw.head())

print("\nTransformed Incremental Data:")
display(clean_inc.head())

# Step 4: Save to transformed/ folder
os.makedirs("transformed", exist_ok=True)
clean_raw.to_csv("transformed/transformed_full.csv", index=False)
clean_inc.to_csv("transformed/transformed_incremental.csv", index=False)


Original Raw Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1,Diana,Tablet,NaN,500.0,2024-01-20,South
1,2,Eve,Laptop,NaN,NaN,2024-04-29,North
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,NaN
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West
4,5,Eve,Tablet,3.0,NaN,2024-03-07,South



Original Incremental Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,101,Alice,Laptop,NaN,900.0,2024-05-09,Central
1,102,NaN,Laptop,1.0,300.0,2024-05-07,Central
2,103,NaN,Laptop,1.0,600.0,2024-05-04,Central
3,104,NaN,Tablet,NaN,300.0,2024-05-26,Central
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North



Transformed Raw Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,tier
0,1,Diana,Tablet,0.0,500.0,2024-01-20,South,0.0,Low
1,2,Eve,Laptop,0.0,0.0,2024-04-29,North,0.0,Low
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,NaN,500.0,Low
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West,1500.0,Medium
4,5,Eve,Tablet,3.0,0.0,2024-03-07,South,0.0,Low



Transformed Incremental Data:


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_price,tier
0,101,Alice,Laptop,0.0,900.0,2024-05-09,Central,0.0,Low
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North,1200.0,Medium
8,109,Grace,Laptop,2.0,600.0,2024-05-29,Central,1200.0,Medium
9,110,Heidi,Phone,0.0,900.0,2024-05-24,NaN,0.0,Low
